In [1]:
# !pip install openai

In [10]:
from openai import OpenAI
import base64
import json

In [3]:
client = OpenAI()

In [4]:
SYSTEM_PROMPT = """
Talk to the user as if they were a pirate.
"""

#### The Basics


In [5]:
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {
            "role": "user",
            "content": "Write a haiku about the color red."
        }
    ]
)

print(completion.choices[0].message)
completion.choices[0].message.content

ChatCompletionMessage(content='Crimson waves do crash,  \nRuby sunsets kiss the sky,  \nHeart of bold adventure.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)


'Crimson waves do crash,  \nRuby sunsets kiss the sky,  \nHeart of bold adventure.'

#### Function Calling

In [9]:
tools = [
  {
      "type": "function",
      "function": {
          "name": "get_weather",
          "parameters": {
              "type": "object",
              "properties": {
                  "location": {"type": "string"}
              },
          },
      },
  }
]

completion = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[{"role": "user", "content": "What's the weather like in Berlin today?"}],
  tools=tools,
)

print(completion.choices[0].message.tool_calls)
completion.choices[0].message.content


[ChatCompletionMessageToolCall(id='call_OlIcXUYqhL9SvMiKHRkXtOmj', function=Function(arguments='{"location":"Berlin"}', name='get_weather'), type='function')]


#### With Vision 👀

In [8]:
# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")


# Path to your image
image_path = "../assets/cole-mader-9WV1S3JsWm8-unsplash.jpg"

# Getting the base64 string
base64_image = encode_image(image_path)

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "What is in this image?",
                },
                {
                    "type": "image_url",
                    "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"},
                },
            ],
        }
    ],
)

# print(response.choices[0])
response.choices[0].message.content


"Arrr, matey! What ye be lookin' at be a mighty fine sight! A vast expanse o' frozen treasure, blanketed by a shroud o' white snow, ‘n surrounded by grand mountains standin' tall like the fierce crew o' a pirate ship! The sky be grumblin' with dark clouds, like a storm brewin' on the horizon. \n\nAye, that icy lake be holdin' secrets beneath its frosty surface! The jagged peaks be kissin' the heavens, while the chill o’ the air cuts sharper than a cutlass! So gather yer crew, and set sail on the adventure o’ nature, fer it be a wild world out there, full o’ wonders awaitin' to be discovered!"

In [18]:
# Define the tools with the expected parameters
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_recommendations",
            "description": "Get recommendations for a song given a song the user likes",
            "parameters": {
                "type": "object",
                "properties": {
                    "song_name": {"type": "string"}
                },
                "required": ["song_name"]
            }
        }
    }
]

# Example message to send to the model
messages = [
    {"role": "user", "content": "Can you recommend a song for me? I like 'Last Christmas' by Wham!"}
]

# Make the API call
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    tools=tools,
)

# Capture the assistant's message
assistant_message = completion.choices[0].message  # Ensure this line captures the assistant's message

# Process tool calls if they exist
if hasattr(assistant_message, 'tool_calls'):
    for tool_call in assistant_message.tool_calls:
        print("Tool call object:", tool_call)  # Inspect the tool_call structure
        
        # Check if the tool_call has the 'arguments' attribute
        if hasattr(tool_call, 'arguments'):
            arguments = tool_call.arguments
            print("Tool call arguments:", arguments)  # Access the arguments directly
            
            # Example of how to use the arguments
            if 'song_name' in arguments:
                song_name = arguments['song_name']
                print(f"Recommended song based on user input: {song_name}")
        else:
            print("Tool call does not have 'arguments' attribute.")
else:
    print("No tool calls found in the assistant's message.")

Tool call object: ChatCompletionMessageToolCall(id='call_ZpesBJhhmfq7DnjjD64yNU2m', function=Function(arguments='{"song_name":"Last Christmas"}', name='get_recommendations'), type='function')
Tool call does not have 'arguments' attribute.
